In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')

### 데이터 전처리

In [ ]:
df = df[df['CustomerID'].notnull()].reset_index(drop=True).copy()
df['CustomerID'] = df['CustomerID'].astype(int)
df = df[df['Quantity'] > 0 ].reset_index(drop=True).copy()
df = df[df['UnitPrice'] > 0 ].reset_index(drop=True).copy()

### 사용자 Retention 분석
- cohort : 특정기간동안 특정경험을 하는 사용자 그룹

In [ ]:
from datetime import datetime

df['Month'] = df['InvoiceDate'].apply(lambda x : datetime(x.year, x.month, 1))

In [ ]:
df['MonthStarted'] = df.groupby(['CustomerID'])['Month'].transform(np.min)

In [ ]:
df.head()

In [ ]:
df['MonthPassed'] = (df['Month'].dt.year - df['MonthStarted'].dt.year) * 12  + (df['Month'].dt.month - df['MonthStarted'].dt.month)

In [ ]:
df.tail()

In [ ]:
cohort = df.groupby(['MonthStarted', 'MonthPassed'])['CustomerID'].apply(lambda x : len(np.unique(x))).reset_index()

In [ ]:
cohort.head()

In [ ]:
cohort = cohort.pivot(index='MonthStarted', columns='MonthPassed')

In [ ]:
cohort

### 재구매율이 떨어지거나 올라간 이유 현상 분석 필요

In [ ]:
cohort = round(cohort.div(cohort.iloc[:, 0], axis=0) * 100,2)
cohort

### Heatmap 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
sns.heatmap(cohort,
            xticklabels = np.arange(0,13),
            yticklabels = [201012, 201101, 201102, 201103, 201104, 201105, 201106,
                           201107, 201108, 201109, 201110, 201111,201112],
           annot=True,
           fmt='.1f',
           ax=ax,
           cmap='YlGnBu')

plt.xlabel('MonthPassed')
plt.show()